In [1]:
%load_ext autoreload
%autoreload 2


%pylab

import os
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

import nltk
!pip3 install seqeval

# Use this module to load the data in a convenient format
from seqeval.metrics import precision_score, recall_score, f1_score, classification_report
from scienceie_loader import load_tokenized_data, load_data_with_char_offsets, get_entity_span_from_B_index

Using matplotlib backend: Qt5Agg
Populating the interactive namespace from numpy and matplotlib


In [2]:
data_root = os.path.join(os.getcwd(), 'original_datasets')
data_train = os.path.join(data_root, 'scienceie2017_train/train2')
data_dev = os.path.join(data_root, 'scienceie2017_dev/dev')
data_test = os.path.join(data_root, 'semeval_articles_test')

In [3]:
# Load the training, dev and test set in tokenized format.
# THIS IS THE RECOMMENDED FORMAT TO USE FOR MOST CASES AS IT'S EASIER TO WORK WITH.

# train_docs, dev_docs and test_docs are lists, where each entry is a document.
# The list entry for each document is a list of tuples, (token, label).
# The labels are BIO tags, wher 'B' and 'I' tags also include the type of entity.

# train_rels, dev_rels, and test_rels are also lists, where each entry corresponds to a document.
# The entry for each document is a list of tuples (label, entity1_start_token_index, entity2_start_token_index).
# Therefore the relations are referred to by the index of their first tokens.

train_docs, train_rels, _ = load_tokenized_data(data_train)
dev_docs, dev_rels, _ = load_tokenized_data(data_dev)
test_docs, test_rels, _ = load_tokenized_data(data_test)

"""
print(f'number of training documents: {len(train_docs)}')
print(f'number of dev documents: {len(dev_docs)}')
print(f'number of test documents: {len(test_docs)}')

print(f'Total number of training relations = {np.sum([len(train_rels_doc) for train_rels_doc in train_rels])}')
print(f'Total number of dev relations = {np.sum([len(dev_rels_doc) for dev_rels_doc in dev_rels])}')
print(f'Total number of test relations = {np.sum([len(test_rels_doc) for test_rels_doc in test_rels])}')
"""

"\nprint(f'number of training documents: {len(train_docs)}')\nprint(f'number of dev documents: {len(dev_docs)}')\nprint(f'number of test documents: {len(test_docs)}')\n\nprint(f'Total number of training relations = {np.sum([len(train_rels_doc) for train_rels_doc in train_rels])}')\nprint(f'Total number of dev relations = {np.sum([len(dev_rels_doc) for dev_rels_doc in dev_rels])}')\nprint(f'Total number of test relations = {np.sum([len(test_rels_doc) for test_rels_doc in test_rels])}')\n"

In [4]:
# the relations for the example documentlen
#len(train_rels)

In [5]:
#print the first entity span
#print(get_entity_span_from_B_index(train_docs, train_rels))
#print(len(train_docs))
# print the second entity span
#print(get_entity_span_from_B_index(train_docs[0], train_rels[0][0][2]))

In [6]:
# THIS FORMAT REQUIRES YOU TO TOKENISE THE TEXT YOURSELF SO IS ONLY RECOMMENDED 
# IF YOUR SEQUENCE LABELLING METHOD REQUIRES A SPECIAL KIND OF TOKENISATION.

# Load the training, dev and test sets as a list where each entry corresponds to one document and contains
# raw text: ["doc1", ...], a list of entity annotations as character offsets: [[(1, 2, entity_index), ...], ...],
# and a list of relations defined by entity IDs: [[(1, 2), ...], ...].

train_docs_chars = load_data_with_char_offsets(data_train)
dev_docs_chars = load_data_with_char_offsets(data_dev)
test_docs_chars = load_data_with_char_offsets(data_test)

"""
print(f'number of training documents: {len(train_docs_chars)}')
print(f'number of dev documents: {len(dev_docs_chars)}')
print(f'number of test documents: {len(test_docs_chars)}')
"""

"\nprint(f'number of training documents: {len(train_docs_chars)}')\nprint(f'number of dev documents: {len(dev_docs_chars)}')\nprint(f'number of test documents: {len(test_docs_chars)}')\n"

In [7]:
test_docs_chars

[['Typical physically-based 2D flood models solve the Shallow Water Equations (SWEs), requiring high computational resources. Many of these models have been developed to obtain better performance, while maintaining the required accuracy, by reducing the complexity of the SWEs. This reduction is usually achieved by approximating or neglecting less significant terms of the equations (Hunter et\xa0al., 2007; Yen and Tsai, 2001). The JFLOW model (Bradbrook et\xa0al., 2004), Urban Inundation Model (UIM) (Chen et\xa0al., 2007), and the diffusive version of LISFLOOD-FP (Hunter et\xa0al., 2005) solve the 2D diffusion wave equations that neglect the inertial (local acceleration) and advection (convective acceleration) terms (Yen and Tsai, 2001). The inertial version of LISFLOOD-FP (Bates et\xa0al., 2010) solves the SWEs without the advection term. In either version of LISFLOOD-FP the flow is decoupled in the Cartesian directions. Other models use the full SWEs but focus on the use of multi reso

In [8]:
train = [j for i in train_docs for j in i]

In [9]:
test = [j for i in test_docs for j in i]

In [10]:
train_dataset = pd.DataFrame(train, columns = ["Words", "Tags"])

In [11]:
test_dataset = pd.DataFrame(test, columns = ["Words", "Tags"])

In [12]:
train_words = train_dataset["Words"].values.tolist()

In [13]:
test_words = test_dataset["Words"].values.tolist()

In [14]:
train_dataset["Tags"].unique()

array(['B-Process', 'I-Process', 'O', 'B-Task', 'I-Task', 'B-Material',
       'I-Material'], dtype=object)

In [15]:
labels = {
    "B-Process" : "Process",
    "I-Process" : "Process", 
    "O" : "O",
    "B-Task" : "Task",
    "I-Task" : "Task",
    "B-Material" : "Material",
    "I-Material" : "Material"
}

In [16]:
train_dataset["Tags"] = train_dataset["Tags"].map(labels)

In [17]:
test_dataset["Tags"] = test_dataset["Tags"].map(labels)

In [18]:
train_dataset.head()

,Words,Tags
0,Two-state,Process
1,models,Process
2,are,O
3,often,O
4,insufficient,O


In [19]:
train_tags = train_dataset["Tags"].values.tolist()

In [20]:
test_tags = test_dataset["Tags"].values.tolist()

In [21]:
train_words = train_dataset["Words"].values.tolist()

In [22]:
test_words = test_dataset["Words"].values.tolist()

In [23]:
char_set = set()
for word in train_words:
    for char in word:
        char_set.add(char)
        
entity_set = set(train_tags)

In [24]:
entity_set

{'Material', 'O', 'Process', 'Task'}

In [25]:
char_set

{'!',
 '$',
 '%',
 '&',
 "'",
 '(',
 ')',
 '*',
 '+',
 ',',
 '-',
 '.',
 '/',
 '0',
 '1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',
 ':',
 ';',
 '<',
 '=',
 '>',
 '?',
 '@',
 'A',
 'B',
 'C',
 'D',
 'E',
 'F',
 'G',
 'H',
 'I',
 'J',
 'K',
 'L',
 'M',
 'N',
 'O',
 'P',
 'Q',
 'R',
 'S',
 'T',
 'U',
 'V',
 'W',
 'X',
 'Y',
 'Z',
 '[',
 ']',
 '^',
 '_',
 'a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'j',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'q',
 'r',
 's',
 't',
 'u',
 'v',
 'w',
 'x',
 'y',
 'z',
 '{',
 '|',
 '}',
 '«',
 '®',
 '¯',
 '°',
 '±',
 'µ',
 '·',
 '»',
 'Å',
 '×',
 'á',
 'é',
 'ö',
 'ü',
 'ń',
 'ʼ',
 '˜',
 '̄',
 'Γ',
 'Δ',
 'Θ',
 'Λ',
 'Ξ',
 'Σ',
 'Τ',
 'Φ',
 'Ψ',
 'Ω',
 'α',
 'β',
 'γ',
 'δ',
 'ε',
 'η',
 'θ',
 'κ',
 'λ',
 'μ',
 'ν',
 'π',
 'ρ',
 'σ',
 'τ',
 'φ',
 'χ',
 'ψ',
 'ω',
 'ϕ',
 'ϵ',
 '\u200b',
 '–',
 '—',
 '‘',
 '’',
 '“',
 '”',
 '•',
 '′',
 '″',
 '⁎',
 '\u2061',
 'ℏ',
 'ℓ',
 '→',
 '⇒',
 '∂',
 '∇',
 '∑',
 '−',
 '∗',
 '∞',
 '∪',
 '∫',
 '∼',
 '≈',
 '≠',


In [26]:
trainer = nltk.tag.hmm.HiddenMarkovModelTrainer(states = entity_set, symbols = char_set)
model = trainer.train_unsupervised(train_words, max_iterations = 25)

iteration 0 logprob -2312286.7452243986
iteration 1 logprob -1431281.8723048365
iteration 2 logprob -1428289.8821977286
iteration 3 logprob -1424807.3826317955
iteration 4 logprob -1420604.8632407424
iteration 5 logprob -1415503.8972358096
iteration 6 logprob -1409466.5038057168
iteration 7 logprob -1402664.838428669
iteration 8 logprob -1395350.7845328774
iteration 9 logprob -1387601.0187392712
iteration 10 logprob -1379431.2001623097
iteration 11 logprob -1371280.0245119336
iteration 12 logprob -1363946.9613118742
iteration 13 logprob -1357905.0117761553
iteration 14 logprob -1353025.9363240595
iteration 15 logprob -1348949.4401228647
iteration 16 logprob -1345445.262096685
iteration 17 logprob -1342414.2062548634
iteration 18 logprob -1339794.243558851
iteration 19 logprob -1337553.811890647
iteration 20 logprob -1335670.0815804887
iteration 21 logprob -1334103.9636530238
iteration 22 logprob -1332811.7585870977
iteration 23 logprob -1331749.2104117365
iteration 24 logprob -1330840.

In [27]:
test_results = model.tag(test_words)

In [28]:
test_results[0 : 5]

[('Typical', 'Process'),
 ('physically-based', 'Process'),
 ('2D', 'Process'),
 ('flood', 'Process'),
 ('models', 'Process')]

In [29]:
test_dataset

,Words,Tags
0,Typical,O
1,physically-based,Process
2,2D,Process
3,flood,Process
4,models,Process
...,...,...
21128,large,O
21129,values,O
21130,of,O
21131,θ,O


In [31]:
type(test_results)

list

In [32]:
test_results_dataset = pd.DataFrame(test_results, columns = ["Words", "Tags"])

In [33]:
test_results_dataset["Tags"].unique()

array(['O'], dtype=object)

In [34]:
train_dataset = pd.DataFrame(train, columns = ["Words", "Tags"])

In [35]:
train_words = train_dataset["Words"].values.tolist()

In [36]:
train_tags = train_dataset["Tags"].values.tolist()train_tags = train_dataset["Tags"].values.tolist()

In [37]:
char_set = set()
for word in train_words:
    for char in word:
        char_set.add(char)
        
entity_set = set(train_tags)

In [38]:
entity_set

{'B-Material', 'B-Process', 'B-Task', 'I-Material', 'I-Process', 'I-Task', 'O'}

In [39]:
trainer = nltk.tag.hmm.HiddenMarkovModelTrainer(states = entity_set, symbols = char_set)
model = trainer.train_unsupervised(train_words, max_iterations = 25)

iteration 0 logprob -2339364.246172905
iteration 1 logprob -1429841.4231726162
iteration 2 logprob -1425354.9354007146
iteration 3 logprob -1420390.1748901904
iteration 4 logprob -1414255.13273574
iteration 5 logprob -1406572.4845784607
iteration 6 logprob -1397506.6230945627
iteration 7 logprob -1387759.5347811168
iteration 8 logprob -1378010.5085839883
iteration 9 logprob -1368502.6380063184
iteration 10 logprob -1359340.2365760275
iteration 11 logprob -1350806.3667667431
iteration 12 logprob -1343124.6804968584
iteration 13 logprob -1336179.0930731033
iteration 14 logprob -1329678.5534595277
iteration 15 logprob -1323537.2067177244
iteration 16 logprob -1317900.752639893
iteration 17 logprob -1312626.5079593016
iteration 18 logprob -1307186.0556657524
iteration 19 logprob -1301075.8039605352
iteration 20 logprob -1294269.1472925243
iteration 21 logprob -1287833.849110978
iteration 22 logprob -1282674.2549827863
iteration 23 logprob -1278625.421027683
iteration 24 logprob -1275409.98

In [40]:
test_results = model.tag(test_words)

In [41]:
test_results_dataset = pd.DataFrame(test_results, columns = ["Words", "Tags"])

In [42]:
test_results_dataset["Tags"].unique()

array(['B-Task'], dtype=object)

In [98]:
train_dataset = pd.DataFrame(train, columns = ["Words", "Tags"])

In [99]:
len(train_dataset)

62475

In [100]:
train_dataset.head()

,Words,Tags
0,Two-state,B-Process
1,models,I-Process
2,are,O
3,often,O
4,insufficient,O


In [101]:
train_dataset["Words"] = train_dataset["Words"].str.lower()

In [102]:
train_dataset

,Words,Tags
0,two-state,B-Process
1,models,I-Process
2,are,O
3,often,O
4,insufficient,O
...,...,...
62470,response,I-Process
62471,[,O
62472,30,O
62473,],O


In [103]:
train_dataset.drop_duplicates(["Words", "Tags"], inplace = True)

In [104]:
train_dataset[train_dataset["Words"] == "another"]

,Words,Tags
5571,another,O
14603,another,B-Process
26401,another,I-Task


In [105]:
train_words = train_dataset["Words"].values.tolist()

In [106]:
train_dataset["Tags"] = train_dataset["Tags"].map(labels)
train_tags = train_dataset["Tags"].values.tolist()

In [107]:
char_set = set()
for word in train_words:
    for char in word:
        char_set.add(char)
        
entity_set = set(train_tags)

In [108]:
train_words

['two-state',
 'models',
 'are',
 'often',
 'insufficient',
 'to',
 'fit',
 'complex',
 'traces',
 ',',
 'therefore',
 'we',
 'also',
 'study',
 'the',
 'approximate',
 'fitting',
 'of',
 'large',
 'm3pps.',
 'in',
 'the',
 'single',
 'class',
 'setting',
 'a',
 'known',
 'limitation',
 'of',
 'mmpps',
 'is',
 'inability',
 'simultaneously',
 'many',
 'statistical',
 'descriptors',
 'due',
 'non-linearity',
 'their',
 'underlying',
 'equations',
 '(',
 'bodrog',
 'heindl',
 'horváth',
 '&',
 'telek',
 '2008',
 ';',
 'gross',
 '2006',
 '2009',
 ')',
 '.',
 'this',
 'has',
 'led',
 'definition',
 'several',
 'approaches',
 'by',
 'composing',
 'multiple',
 'small-sized',
 'mmpps',
 'or',
 'maps',
 'using',
 'kronecker',
 'operators',
 'andersen',
 'nielsen',
 '1998',
 'casale',
 'zhang',
 'smirni',
 '2010',
 '2002',
 'these',
 'methods',
 'employ',
 'composition',
 'for',
 'moment',
 'fitting',
 'offering',
 'different',
 'trade-off',
 'between',
 'computational',
 'cost',
 'and',
 'accu

In [109]:
entity_set

{'Material', 'O', 'Process', 'Task'}

In [110]:
trainer = nltk.tag.hmm.HiddenMarkovModelTrainer(states = entity_set, symbols = char_set)
model = trainer.train_unsupervised(train_words, max_iterations = 1000)

iteration 0 logprob -835924.8079492063
iteration 1 logprob -517482.0186785587
iteration 2 logprob -515966.3661747817
iteration 3 logprob -514545.9910848641
iteration 4 logprob -513262.7632310307
iteration 5 logprob -512028.8903234842
iteration 6 logprob -510608.6976496927
iteration 7 logprob -508775.7158173598
iteration 8 logprob -506415.79902004375
iteration 9 logprob -503690.4872275084
iteration 10 logprob -501001.27909928304
iteration 11 logprob -498635.89762418857
iteration 12 logprob -496649.731282952
iteration 13 logprob -495014.38686630403
iteration 14 logprob -493669.1379157862
iteration 15 logprob -492533.6802605073
iteration 16 logprob -491536.8823534501
iteration 17 logprob -490632.6703947842
iteration 18 logprob -489796.8668137527
iteration 19 logprob -489016.7483372761
iteration 20 logprob -488281.386626033
iteration 21 logprob -487578.57032860204
iteration 22 logprob -486897.78823650815
iteration 23 logprob -486235.37292167067
iteration 24 logprob -485597.7984881936
itera

In [111]:
test_dataset["Tags"] = test_dataset["Tags"].map(labels)

In [135]:
test_real = test_dataset["Tags"]

0          O
1        NaN
2        NaN
3        NaN
4        NaN
        ... 
21128      O
21129      O
21130      O
21131      O
21132      O
Name: Tags, Length: 21133, dtype: object

In [112]:
test_results = model.tag(test_words)

In [113]:
test_results_dataset = pd.DataFrame(test_results, columns = ["Words", "Tags"])

In [114]:
test_results_dataset["Tags"].unique()

array(['O'], dtype=object)

In [120]:
type(test_results)

list

In [121]:
type(test_words)

list

In [125]:
a = []
for i,j in test_results:
    a.append(j)

['Typical',
 'physically-based',
 '2D',
 'flood',
 'models',
 'solve',
 'the',
 'Shallow',
 'Water',
 'Equations',
 '(',
 'SWEs',
 ')',
 ',',
 'requiring',
 'high',
 'computational',
 'resources.',
 'Many',
 'of',
 'these',
 'models',
 'have',
 'been',
 'developed',
 'to',
 'obtain',
 'better',
 'performance',
 ',',
 'while',
 'maintaining',
 'the',
 'required',
 'accuracy',
 ',',
 'by',
 'reducing',
 'the',
 'complexity',
 'of',
 'the',
 'SWEs.',
 'This',
 'reduction',
 'is',
 'usually',
 'achieved',
 'by',
 'approximating',
 'or',
 'neglecting',
 'less',
 'significant',
 'terms',
 'of',
 'the',
 'equations',
 '(',
 'Hunter',
 'et',
 'al.',
 ',',
 '2007',
 ';',
 'Yen',
 'and',
 'Tsai',
 ',',
 '2001',
 ')',
 '.',
 'The',
 'JFLOW',
 'model',
 '(',
 'Bradbrook',
 'et',
 'al.',
 ',',
 '2004',
 ')',
 ',',
 'Urban',
 'Inundation',
 'Model',
 '(',
 'UIM',
 ')',
 '(',
 'Chen',
 'et',
 'al.',
 ',',
 '2007',
 ')',
 ',',
 'and',
 'the',
 'diffusive',
 'version',
 'of',
 'LISFLOOD-FP',
 '(',
 'Hu

In [128]:
test_results_dataset.head()

,Words,Tags
0,Typical,O
1,physically-based,O
2,2D,O
3,flood,O
4,models,O


In [129]:
test_results_f1 = test_results_dataset["Tags"]

In [131]:
type(test_results_f1)

pandas.core.series.Series

In [132]:
test_results_f1 = list(test_results_f1)

In [134]:
f1_score()

['O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O'